In [68]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path

'''
readCSV :CSV 파일을 읽는다. 고정포맷이 정해짐 
trainModel(ts =0.25): 입력된 모델에서 학습시킨다. ts 테스트 케이스 
predict(): 학습시킨 모델에 test데이터가 입력되면 예측한다. 
writePredictionResult(): 예측 결과를 파일로 출력한다. 
'''
class FilterReviewSV:
    def __init__(self):
        
        self.model_filename = 'model/FilterReviewSV_model'
        self.isSetModel = False
        if not(self.isSetModel) and os.path.isfile(self.model_filename+'.sav'):
            print('모델을 불러오는중...')
            self.gnb = pickle.load(open(self.model_filename+'.sav', 'rb'))
            
    def input_X(self, para):
        pre = Preprocessor()
        print(para)
        pre.setPara(para)
        print(pre.sentences)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result
        
    def readCSV(self,filename,tag=True):
        self.data = pd.read_csv(filename,engine='python')
        self.X_content = self.data['sentence'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['label'].values)
        
    def trainModel(self,ts):
        self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        self.gnb = GaussianNB()
        self.y_pred = self.gnb.fit(self.X_train, self.y_train).predict(self.X_test)

        # 테스트 socre 확인
        print("Number of right labeled points out of a total %d points : %.2f %%"
      % (len(self.y_test),100*(len(self.y_test)-(self.y_test != self.y_pred).sum())/len(self.y_test)))
        
        pickle.dump(self.gnb, open(self.model_filename+'.sav', 'wb'))
        isSetModel = True
    def predict(self):
        self.y_pred = self.gnb.predict(self.X)
        self.df = pd.DataFrame({'p_label':self.y_pred, 'sentence':self.X_content})
        return self.y_pred
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='utf-8-sig')
        


In [69]:
fr = FilterReviewSV()
fr.readCSV(tag=True,filename='Data_ReviewFilter/TrainData.csv')
fr.trainModel(ts=0.25)

#fr.readCSV(tag=False,filename='Test.csv')
#fr.predict()

#fr.writePredictionResult('sample.csv')
#print(fr.y_pred)

모델을 불러오는중...
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '수학을 거의 반영하지 않는 곳에 가려고 하는데 읽어 주셔서 감사합니다'
 '친 절한 답변 부탁합니다' '예비고 3 질문']
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '수학을 거의 반영하지 않는 곳에 가려고 하는데 읽어 주셔서 감사합니다'
 '친 절한 답변 부탁합니다' '예비고 3 질문']


RuntimeError: No matching overloads found for tokenize in find. at native\common\jp_method.cpp:127